In [3]:
! pip install pymongo scikit-learn python-dotenv

   ---------------------------------------- 0.0/680.4 kB ? eta -:--:--
    --------------------------------------- 10.2/680.4 kB ? eta -:--:--
   -- ------------------------------------ 41.0/680.4 kB 495.5 kB/s eta 0:00:02
   ----- --------------------------------- 92.2/680.4 kB 871.5 kB/s eta 0:00:01
   ---------- ----------------------------- 174.1/680.4 kB 1.1 MB/s eta 0:00:01
   ------------------ --------------------- 307.2/680.4 kB 1.5 MB/s eta 0:00:01
   ----------------------------- ---------- 501.8/680.4 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 680.4/680.4 kB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   --------------------------------------- 307.7/307.7 kB 19.8 MB/s eta 0:00:00


# Import Libraries


In [18]:
import tensorflow as tf
import pandas as pd
import numpy as mp
from tensorflow.keras import layers
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import models
from keras.optimizers import Adam

# Connect with MongoDB 

In [ ]:
MONGO_CONNECTION_STRING = ''
clint = MongoClient(MONGO_CONNECTION_STRING)
db = clint['traffic sign']
train_collection = db['train_collection']
validation_collection = db['validation_collection']

data = list(collection.find())
df = pd.DataFrame(data)

df.drop('_id', axis=1, inplace=True)
df.head()

# Task 1

## Load the data and preprocessing

In [23]:
batch_size = 64
img_height = 64
img_width = 64
data = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_image = data.flow_from_directory(
    r'D:/T5/T5_Week4_MProj/traffic_sign_dataset/train',
    #r'C:\Users\SaadL\OneDrive\المستندات\for_VS\T5_Week4_MProj\traffic_sign_dataset\train',
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode='categorical',
    subset='training'
)

validation_image = data.flow_from_directory(
    r'D:/T5/T5_Week4_MProj/traffic_sign_dataset/train',
    #r'C:\Users\SaadL\OneDrive\المستندات\for_VS\T5_Week4_MProj\traffic_sign_dataset\train',
    target_size = (img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 3465 images belonging to 2 classes.
Found 866 images belonging to 2 classes.


## Data Augmentation

In [24]:
Data_generator_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 40,
    width_shift_range=0.2,
    height_shift_range =0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearset'
)

# Task 2

## Building the CNN model

In [25]:
traffic_model = models.Sequential([
# Layer 1
layers.Conv2D(64, (5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=(224, 224, 3)),
layers.MaxPooling2D((2, 2)),
layers.BatchNormalization(),
layers.Conv2D(128, (5, 5), strides=(1, 1), padding='same', activation='relu'),
layers.MaxPooling2D((2, 2)),
layers.Dropout((0.3)),
layers.Conv2D(256, (5, 5), strides=(1, 1), padding='same', activation='relu'),
layers.BatchNormalization(),
# Layer 2
layers.Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'),
layers.MaxPooling2D((2, 2)),
layers.BatchNormalization(),
layers.Dropout((0.3)),
layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'),
layers.MaxPooling2D((2, 2)),
layers.BatchNormalization(),
# Layer 3
layers.Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'),
layers.MaxPooling2D((2, 2)),
layers.BatchNormalization(),
layers.Dropout((0.3)),
layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'),
layers.MaxPooling2D((2, 2)),
layers.BatchNormalization(),
# Layer 4
layers.Flatten(),
layers.Dense(units=256, activation='relu'),
layers.Dropout((0.3)),
layers.Dense(units=15, activation='softmax')
])

d:\Anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Compiling the traffic_model + summary

In [26]:
traffic_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
traffic_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_29 (Conv2D)              │ (None, 224, 224, 64)   │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_24          │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 112, 112, 128)  │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 56, 56, 256)    │       819,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_25          │ (None, 56, 56, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 56, 56, 128)    │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_26          │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 14, 14, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 14, 14, 128)    │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 7, 7, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 3, 3, 256)      │         1,024 │
│ (BatchNormalization)            │                        │             

 Total params: 2,807,951 (10.71 MB)

 Trainable params: 2,805,775 (10.70 MB)

 Non-trainable params: 2,176 (8.50 KB)

## Fitting the traffic_model with the training and validating the training

In [28]:
traffic_history = traffic_model.fit(Data_generator_train.flow(train_image), batch_size=400, epochs=100, validation_data=(validation_image))

# Task 3

## Testing the model with unseen data

In [ ]:
traffic_model.evaluate(x_test)